Will create two maps that show incidents of Chicago from 2001 to Present. 

The dataset was used in a course of IBM Data Science Professional Certificate, and was modified to a smaller size. 

Original data is available from Chicago Data Portal: 
https://data.cityofchicago.org/Public-Safety/Crimes-2020/qzdf-xmn8

In [1]:
import numpy as np  
import pandas as pd 

!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Folium installed and imported!')

Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\micha\anaconda3

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.2               |     pyhd8ed1ab_0          26 KB  conda-forge
    certifi-2020.12.5          |   py38haa244fe_1         144 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         857 KB

The following NEW packages will be INSTALLED:

  altair             conda-forge

In [2]:
df_incidents = pd.read_csv('Chicago_Crime.csv')

df_incidents.head()

,ID,CASE_NUMBER,DATE,BLOCK,IUCR,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,ARREST,DOMESTIC,...,WARD,COMMUNITY_AREA_NUMBER,FBICODE,X_COORDINATE,Y_COORDINATE,YEAR,UPDATEDON,LATITUDE,LONGITUDE,LOCATION
0,3512276,HK587712,08/28/2004 05:50:56 PM,047XX S KEDZIE AVE,890,THEFT,FROM BUILDING,SMALL RETAIL STORE,False,False,...,14.0,58.0,6,1155838.0,1873050.0,2004,02/10/2018 03:50:01 PM,41.807441,-87.703956,"(41.8074405, -87.703955849)"
1,3406613,HK456306,06/26/2004 12:40:00 PM,009XX N CENTRAL PARK AVE,820,THEFT,$500 AND UNDER,OTHER,False,False,...,27.0,23.0,6,1152206.0,1906127.0,2004,02/28/2018 03:56:25 PM,41.898280,-87.716406,"(41.898279962, -87.716405505)"
2,8002131,HT233595,04/04/2011 05:45:00 AM,043XX S WABASH AVE,820,THEFT,$500 AND UNDER,NURSING HOME/RETIREMENT HOME,False,False,...,3.0,38.0,6,1177436.0,1876313.0,2011,02/10/2018 03:50:01 PM,41.815933,-87.624642,"(41.815933131, -87.624642127)"
3,7903289,HT133522,12/30/2010 04:30:00 PM,083XX S KINGSTON AVE,840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,False,False,...,7.0,46.0,6,1194622.0,1850125.0,2010,02/10/2018 03:50:01 PM,41.743665,-87.562463,"(41.743665322, -87.562462756)"
4,10402076,HZ138551,02/02/2016 07:30:00 PM,033XX W 66TH ST,820,THEFT,$500 AND UNDER,ALLEY,False,False,...,15.0,66.0,6,1155240.0,1860661.0,2016,02/10/2018 03:50:01 PM,41.773455,-87.706480,"(41.773455295, -87.706480471)"


In [3]:
df_incidents.shape

(533, 22)

In [6]:
# Any rows that do not have values in LATITUDE and LONGITUDE columns must be deleted in order to make a map. 
# Replace empty values with NaN

df_incidents.replace("", np.nan, inplace = True)

In [7]:
# Count the number of NaN in each column

df_incidents.isnull().sum()

ID                        0
CASE_NUMBER               0
DATE                      0
BLOCK                     0
IUCR                      0
PRIMARY_TYPE              0
DESCRIPTION               0
LOCATION_DESCRIPTION      0
ARREST                    0
DOMESTIC                  0
BEAT                      0
DISTRICT                  0
WARD                     43
COMMUNITY_AREA_NUMBER    43
FBICODE                   0
X_COORDINATE              4
Y_COORDINATE              4
YEAR                      0
UPDATEDON                 0
LATITUDE                  4
LONGITUDE                 4
LOCATION                  4
dtype: int64

In [9]:
# Drop rows that have NaN values in LATITUDE and LONGITUDE columns 

df_incidents.dropna(subset=["LATITUDE", "LONGITUDE"], axis=0, inplace=True)

df_incidents.reset_index(drop=True, inplace=True)

# Check the shape of the df 

df_incidents.shape

(529, 22)

In [26]:
latitude = 41.8781
longitude = -87.6298

chicago_map = folium.Map(location=[latitude, longitude], zoom_start=12)

chicago_map

In [27]:
# Create a simple map that shows incidents in Chicago

incidents = folium.map.FeatureGroup()

for lat, lng, in zip(df_incidents.LATITUDE, df_incidents.LONGITUDE):
    incidents.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=6, # Define the size of the circles
            color='red',
            fill=True,
            fill_color='#cc31cc',
            fill_opacity=0.6
          
        )
    )


chicago_map.add_child(incidents)

In [30]:
# Add a second map that shows the types of the incidents (if you click a circle, a description will appear)

chicago_map2 = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lng, label in zip(df_incidents.LATITUDE, df_incidents.LONGITUDE, df_incidents.PRIMARY_TYPE):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5, 
        color='red',
        fill=True,
        popup=label,
        fill_color='#cc31cc',
        fill_opacity=0.6
    ).add_to(chicago_map2)


chicago_map2